In [1]:
import os
import sys

from arcgis.gis import GIS
from arcgis.geometry import Point, distance
from dotenv import load_dotenv
from tqdm.notebook import tqdm

sys.path.append('../')

from water_reach_tools import Reach
from water_reach_tools.water_reach_tools import ReachLineFeatureLayer, ReachPointFeatureLayer

%load_ext autoreload
%autoreload 2

In [2]:
load_dotenv('../.env')

True

In [3]:
gis = GIS(
    url=os.getenv('ARCGIS_URL', 'https://arcgis.com'),
    username=os.getenv('ARCGIS_USERNAME', None),
    password=os.getenv('ARCGIS_PASSWORD', None)
)
gis

GIS @ https://knu2xs.maps.arcgis.com version:8.2

In [4]:
line_lyr_id=os.getenv('REACH_LINE_ID')
centroid_lyr_id=os.getenv('REACH_CENTROID_ID')

line_lyr = ReachLineFeatureLayer.from_item_id(gis, line_lyr_id)
centroid_lyr = ReachPointFeatureLayer.from_item_id(gis, centroid_lyr_id)

In [5]:
reach_id_lst = line_lyr.get_unique_values('reach_id', 'gauge_id IS NOT NULL')
len(reach_id_lst)

517

In [6]:
def update_reach_stage(rid):
    rch = Reach.get_from_aw(rid)
    line_lyr.update_attributes_only(rch)
    centroid_lyr.update_attributes_only(rch)

In [16]:
update_reach_stage(reach_id_lst[0])

pass in features as list of Features, dicts or PropertyMap
Parameters not valid for edit_features
pass in features as list of Features, dicts or PropertyMap
Parameters not valid for edit_features


In [7]:
from dask.distributed import Client
import dask.bag
import dask.config

In [8]:
dask.config.set(scheduler='threads')

In [9]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:64381 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [10]:
db = dask.bag.from_sequence(reach_id_lst).map(update_reach_stage)
db

dask.bag<update_reach_stage, npartitions=104>

In [19]:
db.map(update_reach_stage)

dask.bag<update_reach_stage, npartitions=104>

In [6]:
fs = line_lyr.query_by_reach_id(reach_id_lst[50])
fs

<FeatureSet> 1 features

In [7]:
df = fs.sdf
df

,OBJECTID,reach_name,river_name,river_name_alternate,error,notes,difficulty,difficulty_minimum,difficulty_maximum,difficulty_outlier,...,difficulty_filter,gauge_stage,tracing_method,gauge_update_datetime,putin_x,putin_y,takeout_x,takeout_y,gauge_units,SHAPE
0,8946,Section 3 - Hwy. 392 to Haddens Ferry,Toro Bayou,,None,,II+,None,II+,None,...,2.2,too low,None,NaT,-93.5444,31.2044,-93.535,31.1859,ft,"{""paths"": [[[-93.5448418792508, 31.20422310255..."


In [1]:
from arcgis.mapping import WebMap

/Users/knu2xs/projects/water-reach-tools/venv/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [2]:
WebMap()

MapView(hide_mode_switch=True, layout=Layout(height='400px', width='100%'))